In [563]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import scipy.stats as st

In [564]:
paths = glob('data/sensors*.csv')
data  = [pd.read_csv(p, index_col=0, parse_dates=True) for p in paths]
keys  = [''.join(p.split('/')[1].replace('sensors_','').split('_')[:2]) for p in paths]
data  = pd.concat(data, keys=keys)
data = data.swaplevel().unstack(0).T.swaplevel().unstack()

data = data.interpolate(kind='cubic')
data.to_csv('data/HOBOs_ICHAN_20220807_20230506.csv')

In [565]:
vientoMARPCH = pd.read_csv('data/Viento_MARPCH.csv', index_col=0, parse_dates=True)['2022':]
vientoICHAN  = pd.read_csv('data/Viento_ICHAN.csv', index_col=0, parse_dates=True)['2022':]

vMARPCH = -vientoMARPCH.iloc[:,0]*np.cos(np.deg2rad(vientoMARPCH.iloc[:,1]))
vICHAN  = -vientoICHAN.iloc[:,0]*np.cos(np.deg2rad(vientoICHAN.iloc[:,1]))
period = slice('2022-11','2023-01')
x = vMARPCH[period].dropna().resample('d').mean()
y = vICHAN[period].resample('d').mean().reindex(x.index).dropna()
x = x.reindex(y.index)
m = st.linregress(x,y)

vICHAN_new = (vMARPCH*m.slope + m.intercept)
vICHAN_new = vICHAN_new.resample('30min').interpolate()
vICHAN_new.name = 'VientoMeridional_ms-1'
vICHAN_new.to_csv('data/VientoMeridional_ICHAN.csv')

In [566]:
mask = (vICHAN_new.resample('d').mean()>5.0)
for i in range(1,len(mask)-1):
    if mask.iloc[i] == True:
        if mask.iloc[i-1] == False:
            if mask.iloc[i+1] == False:
                mask.iloc[i] = False
    # if mask[i] == False:
    #     if mask[i-1] == True:
    #         if mask[i+1] == True:
    #             mask[i] = True
mask = mask.resample('h').ffill().reindex(vICHAN_new.index).ffill()
groupN = mask.diff().cumsum().fillna(0)
groupN = groupN.where(groupN%2==0).dropna()

x = pd.DataFrame(mask.copy())
x['groupN'] = groupN
events = []
for i, g in x.groupby(['groupN']):
    inicio = g.index[0].strftime('%F')
    fin    = g.index[-1].strftime('%F')
    events.append((inicio,fin))
events = pd.DataFrame(events, columns=['Inicio','Fin'])
events_slices = [slice(x,y) for x,y in zip(events.Inicio, events.Fin)]
events.to_csv('data/EventosSurgencia_segunViento.csv')

In [567]:
MaxTempTiming = []
for i in range(len(events_slices)):
    try:
        x = data['synop10m'][events_slices[i]].iloc[:,0]
        timing = x.idxmax()
        MaxTempTiming.append(timing)
    except:
        pass
eventsTemp = [(time-pd.Timedelta(days=1), time+pd.Timedelta(days=5)) for time in MaxTempTiming]
eventsTemp = [(e[0].strftime('%F %H:%M:%S'),e[1].strftime('%F %H:%M:%S')) for e in eventsTemp]
eventsTemp = pd.DataFrame(eventsTemp, columns=['Inicio','Fin'])
eventsTemp.to_csv('data/EventosSurgencia_segunTemperatura.csv')

In [568]:
groups10m = [data['synop10m'].loc[MaxTempTiming[i]-pd.Timedelta(days=1):MaxTempTiming[i]+pd.Timedelta(days=5)].reset_index(drop=True)
             for i in range(len(MaxTempTiming))]
groups30m = [data['synop30m'].loc[MaxTempTiming[i]-pd.Timedelta(days=1):MaxTempTiming[i]+pd.Timedelta(days=5)].reset_index(drop=True)
             for i in range(len(MaxTempTiming))]
mean10m = pd.concat(groups10m, keys=MaxTempTiming).unstack().T.mean(axis=1).unstack().T
std10m  = pd.concat(groups10m, keys=MaxTempTiming).unstack().T.std(axis=1).unstack().T

mean30m = pd.concat(groups30m, keys=MaxTempTiming).unstack().T.mean(axis=1).unstack().T
std30m  = pd.concat(groups30m, keys=MaxTempTiming).unstack().T.std(axis=1).unstack().T


index = np.linspace(-1*24, 5*24, len(mean10m))
mean10m.index = index
std10m.index = index
mean30m.index = index
std30m.index = index

means = pd.concat([mean10m, mean30m], keys=['10m','30m'])
stds  = pd.concat([std10m, std30m], keys=['10m','30m'])

composite = pd.concat([means,stds], keys=['means','stds'])
composite.unstack().T.to_csv('data/CompuestosSurgencia_Temperatura_pH_Oxigeno.csv')